# Netmiko & Diffing

## Prepare Your Environment

Spin up a new virtual environment using your tool of choice (`virtualenv`, `virtualenvwrapper`, etc.) and install a number of libraries:

```
pip install netmiko ncclient jupyter
```

Then run up `jupyter-notebook` and open this file.

## Connection Variables

In [2]:
# Local CSR 1000v (running under vagrant)
HOST = '127.0.0.1'
PORT_NC = 2223
PORT_SSH = 2222
USER = 'vagrant'
PASS = 'vagrant'

# used for netmiko device connection
PLATFORM = 'cisco_ios'

## Connect both netmiko and ncclient

In [3]:
from netmiko import ConnectHandler
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)
d = ConnectHandler(device_type=PLATFORM, ip=HOST, port=PORT_SSH, username=USER, password=PASS)

prompt = d.find_prompt()

## Capture **Initial** Netconf Config

In [5]:
retval = m.get_config(source='running', filter=('xpath', '/native'))
initial_netconf = retval.data_ele[0]

## Sample Config To Apply

In [10]:
config = '''ip access-list standard FOO
 permit 10.128.0.0 0.127.255.255
 permit 10.119.120.0 0.0.7.255
 permit 10.87.79.0 0.0.0.255
 permit 10.87.96.0 0.0.0.255
 deny   any
'''

# config = '''no ip access-list standard FOO'''

## Copy Config To Flash (IOS-XE Only)

In [5]:
d.remote_conn.sendall('copy running-config flash:backup123.cfg\n')
get_reply(d.remote_conn, ']? ')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, prompt)

## Apply Config

In [11]:
output = d.send_config_set(config.splitlines())

## Capture Running Config

In [7]:
running_config_before = d.send_command('show running-config')

## Capture Netconf Config

In [8]:
retval = m.get_config(source='running', filter=('xpath', '/native'))
native = retval.data_ele[0]
reapply = '<config>' + etree.tostring(native).decode() + '</config>'

## Diff Netconf Config Now With Initial Netconf Config

In [9]:
initial_xml_str = etree.tostring(initial_netconf, pretty_print=True)
after_xml_str = etree.tostring(native, pretty_print=True)

In [10]:
from difflib import context_diff

print('\n'.join(context_diff(
    initial_xml_str.decode().splitlines(),
    after_xml_str.decode().splitlines())))

*** 

--- 

***************

*** 62,67 ****

--- 62,116 ----

          <GigabitEthernet>1</GigabitEthernet>
        </source-interface>
      </tftp>
+     <access-list>
+       <standard xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-acl">
+         <name>FOO</name>
+         <access-list-seq-rule>
+           <sequence>10</sequence>
+           <permit>
+             <std-ace>
+               <ipv4-prefix>10.128.0.0</ipv4-prefix>
+               <mask>0.127.255.255</mask>
+             </std-ace>
+           </permit>
+         </access-list-seq-rule>
+         <access-list-seq-rule>
+           <sequence>20</sequence>
+           <permit>
+             <std-ace>
+               <ipv4-prefix>10.119.120.0</ipv4-prefix>
+               <mask>0.0.7.255</mask>
+             </std-ace>
+           </permit>
+         </access-list-seq-rule>
+         <access-list-seq-rule>
+           <sequence>30</sequence>
+           <permit>
+             <std-ace>
+               <ipv4-prefix>10.87.79

## Restore Config From Flash

In [11]:
d.remote_conn.sendall('configure replace flash:backup123.cfg\n')
output = get_reply(d.remote_conn, '[no]: ')
print(output)
d.remote_conn.sendall('y\n')
output = get_reply(d.remote_conn, prompt)
print(output)

configure replace flash:backup123.cfg
This will apply all necessary additions and deletions
to replace the current running configuration with the
contents of the specified configuration file, which is
assumed to be a complete configuration, not a partial
configuration. Enter Y if you are sure you want to proceed. ? [no]: 
y
Total number of passes: 1
Rollback Done

csr1kv#


## Reapply Netconf Config

In [12]:
m.edit_config(reapply, format='xml', target='running', default_operation='merge')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:e5087d46-bfd7-44a8-ab0f-45b0cb73ca07" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

## Get Running Config Again

In [13]:
running_config_after = d.send_command('show running-config')

## Diff Before & After

Any difference here implies a lack of coverage.

In [14]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print(''.join(context_diff(running_config_before.splitlines()[5:], running_config_after.splitlines()[5:])))

## Delete Backup From Flash

In [15]:
d.remote_conn.sendall('delete flash:backup123.cfg\n')
output = get_reply(d.remote_conn, ']? ')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, '[confirm]')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, prompt)

## Tidyup Sessions

In [16]:
d.disconnect()
m.close_session()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:b15a00d4-a68f-4f74-a3d1-4d02401370b2" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>